In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random
import requests

%matplotlib inline

/home/ihsaan/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
url =  "https://www.mit.edu/~ecprice/wordlist.10000"
words = response = requests.get(url).text.splitlines()
random.sample(words, 10)

['wire',
 'equal',
 'vendor',
 'rentals',
 'techno',
 'grad',
 'pursue',
 'gordon',
 'carter',
 'headed']

In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

In [4]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([60647, 3]) torch.Size([60647])
torch.Size([7759, 3]) torch.Size([7759])
torch.Size([7474, 3]) torch.Size([7474])


In [5]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff} | my {dt.shape} | py {t.shape}')

In [6]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility

C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1


parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [7]:
n_batch = 32
n = n_batch

# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (n_batch,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] 

In [54]:
Xb.shape, Xb

(torch.Size([32, 3]),
 tensor([[20,  9, 20],
         [16,  5, 14],
         [ 5, 12,  2],
         [ 0,  6, 18],
         [ 2, 18, 15],
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 2, 18, 15],
         [20, 21, 18],
         [12,  9, 20],
         [10, 15, 21],
         [15, 16, 15],
         [ 0, 13,  9],
         [12, 15, 12],
         [ 1,  7,  7],
         [19, 20,  5],
         [23,  9, 18],
         [ 0,  0,  3],
         [ 9, 14, 20],
         [ 5, 22, 15],
         [ 0,  0, 14],
         [ 0,  0, 19],
         [ 0,  0,  4],
         [ 8,  5,  4],
         [ 0,  0,  0],
         [ 9, 14,  7],
         [ 0,  0,  0],
         [ 6, 12,  5],
         [24,  1, 13],
         [ 0,  0,  0],
         [13,  1, 14],
         [15, 16,  5]]))

In [8]:
# forward pass

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.2952, grad_fn=<NegBackward0>)

In [22]:
for t in [bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
        print(t.shape)

torch.Size([32, 64])
torch.Size([1, 64])
torch.Size([1, 64])
torch.Size([32, 64])
torch.Size([32, 64])
torch.Size([32, 64])
torch.Size([1, 64])
torch.Size([32, 30])
torch.Size([32, 3, 10])


In [32]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n
dprobs = dlogprobs * (1.0 / probs)
dcounts_sum_inv = (dprobs * counts).sum(1, keepdim=True)
dcounts_sum = (- counts_sum **-2) * dcounts_sum_inv
dcounts = counts_sum_inv * dprobs
dcounts += dcounts_sum * torch.ones_like(counts)
dnorm_logits = counts * dcounts
dlogit_maxes =  - dnorm_logits.sum(1, keepdim=True)
dlogits = dnorm_logits
dlogits += dlogit_maxes * F.one_hot(logits.max(1).indices, num_classes=logits.shape[1])
dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)
dhpreact = (1.0 - h**2) * dh
dbngain = (dhpreact * bnraw).sum(0, keepdim=True)
dbnbias = dhpreact.sum(0, keepdim=True)
dbnraw = (dhpreact * bngain)
dbndiff = bnvar_inv * dbnraw
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
dbnvar = (-0.5*(bnvar + 1e-5)**-1.5)* dbnvar_inv
dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
dbndiff += (2*bndiff) * dbndiff2
dhprebn = dbndiff.clone()
dbnmeani = (-dbndiff).sum(0)
dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
demb = dembcat.view(emb.shape)
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
  for j in range(Xb.shape[1]):
    ix = Xb[k,j]
    dC[ix] += demb[k,j]

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0 | my torch.Size([32, 27]) | py torch.Size([32, 27])
probs           | exact: True  | approximate: True  | maxdiff: 0.0 | my torch.Size([32, 27]) | py torch.Size([32, 27])
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0 | my torch.Size([32, 1]) | py torch.Size([32, 1])
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0 | my torch.Size([32, 1]) | py torch.Size([32, 1])
counts          | exact: True  | approximate: True  | maxdiff: 0.0 | my torch.Size([32, 27]) | py torch.Size([32, 27])
norm_logits     | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09 | my torch.Size([32, 27]) | py torch.Size([32, 27])
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0 | my torch.Size([32, 1]) | py torch.Size([32, 1])
logits          | exact: True  | approximate: True  | maxdiff: 0.0 | my torch.Size([32, 27]) | py torch.Size([32, 27])
h               | exact: True  | app

In [11]:
# backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.295159339904785 diff: 4.76837158203125e-07


In [33]:
# backward pass

# for each set of logits in batch dlogits = p_

dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n

cmp('logits', dlogits, logits) 

logits          | exact: False | approximate: True  | maxdiff: 6.752088665962219e-09 | my torch.Size([32, 27]) | py torch.Size([32, 27])


In [ ]:
logits.shape, Yb.shape

In [46]:
F.softmax(logits, 1)[0].sum()

tensor(1., grad_fn=<SumBackward0>)

In [47]:
dlogits[0] * n

tensor([ 0.0471,  0.0414,  0.0236,  0.0210,  0.0237,  0.0699,  0.0435,  0.0154,
         0.0146,  0.0488,  0.0959,  0.0247,  0.0738,  0.0135,  0.0571,  0.0184,
         0.0245,  0.0132,  0.0258,  0.0125,  0.0152, -0.9612,  0.0239,  0.0823,
         0.0635,  0.0392,  0.0290], grad_fn=<MulBackward0>)